In [2]:
import requests
import json
import http.client
import time
from datetime import datetime, timedelta
import re

# Constants
AIRTABLE_BASE_ID = 'app7mcJMf9O8GQRal'
AIRTABLE_API_KEY = ''
YOUTUBE_TABLE = 'youtube_user'
POSTS_TABLE = 'youtube_post'
RAPIDAPI_KEY = ""

def get_youtube_users():
    """Lấy danh sách username từ bảng youtube_user"""
    url = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/{YOUTUBE_TABLE}"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_API_KEY}',
        'Content-Type': 'application/json'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return [record['fields'].get('username') for record in data.get('records', [])]
    return []

def get_existing_video_ids():
    """Lấy danh sách video_id đã tồn tại"""
    url = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/{POSTS_TABLE}"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_API_KEY}',
        'Content-Type': 'application/json'
    }
    
    existing_ids = set()
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        existing_ids = {record['fields'].get('video_id') for record in data.get('records', [])}
    return existing_ids

def get_channel_videos(channel_id):
    """Lấy tất cả video từ một channel"""
    videos = []
    next_token = None
    
    while True:
        conn = http.client.HTTPSConnection("youtube-media-downloader.p.rapidapi.com")
        headers = {
            'x-rapidapi-key': RAPIDAPI_KEY,
            'x-rapidapi-host': "youtube-media-downloader.p.rapidapi.com"
        }
        
        url = f"/v2/channel/videos?channelId={channel_id}&type=videos&sortBy=newest"
        if next_token:
            url += f"&nextToken={next_token}"
            
        try:
            conn.request("GET", url, headers=headers)
            res = conn.getresponse()
            data = json.loads(res.read().decode("utf-8"))
            
            if not data.get("status"):
                print(f"Lỗi khi lấy video từ channel {channel_id}: {data.get('errorId')}")
                break
                
            if "items" in data:
                videos.extend(data["items"])
                
            next_token = data.get("nextToken")
            if not next_token:
                break
                
            time.sleep(1)  # Tránh rate limit
            
        except Exception as e:
            print(f"Lỗi: {str(e)}")
            break
        finally:
            conn.close()
            
    return videos

def convert_length_to_time(length_text):
    """Convert length text (e.g. "10:28") to seconds"""
    if not length_text:
        return 0
        
    parts = length_text.split(':')
    
    if len(parts) == 2:  # mm:ss
        minutes = int(parts[0])
        seconds = int(parts[1])
        return minutes * 60 + seconds
    elif len(parts) == 3:  # hh:mm:ss
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        return hours * 3600 + minutes * 60 + seconds
        
    return 0

def extract_view_count(view_text):
    """Extract number from view count text (e.g. "7,138 views")"""
    if not view_text:
        return 0
    number = re.sub(r'[^0-9,]', '', view_text)
    return int(number.replace(',', ''))

def convert_published_time(time_text):
    """Convert relative time to actual date"""
    now = datetime.now()
    
    if not time_text:
        return now.strftime('%Y-%m-%d')
        
    match = re.search(r'(\d+)\s*(hour|day|week|month|year)s?\s*ago', time_text.lower())
    if not match:
        return now.strftime('%Y-%m-%d')
        
    number = int(match.group(1))
    unit = match.group(2)
    
    if unit == 'hour':
        date = now - timedelta(hours=number)
    elif unit == 'day':
        date = now - timedelta(days=number)
    elif unit == 'week':
        date = now - timedelta(weeks=number)
    elif unit == 'month':
        date = now - timedelta(days=number*30)  # approximate
    elif unit == 'year':
        date = now - timedelta(days=number*365)  # approximate
    else:
        return now.strftime('%Y-%m-%d')
        
    return date.strftime('%Y-%m-%d')

def main():
    # Lấy danh sách username
    usernames = get_youtube_users()
    print(f"Tìm thấy {len(usernames)} users")
    
    # Lấy danh sách video_id đã tồn tại
    existing_videos = get_existing_video_ids()
    print(f"Đã có {len(existing_videos)} videos trong database")
    
    # Xử lý từng username
    for username in usernames:
        print(f"\nĐang xử lý channel: {username}")
        videos = get_channel_videos(username)
        
        # Lọc các video chưa tồn tại
        new_videos = [v for v in videos if v['id'] not in existing_videos]
        print(f"Tìm thấy {len(new_videos)} videos mới")
        
        # Thêm các video mới vào Airtable
        headers = {
            'Authorization': f'Bearer {AIRTABLE_API_KEY}',
            'Content-Type': 'application/json'
        }
        
        for video in new_videos:
            data = {
                "fields": {
                    "video_id": video['id'],
                    "title": video['title'],
                    "length": convert_length_to_time(video.get('lengthText', '')),
                    "viewCount": extract_view_count(video.get('viewCountText', '')),
                    "published_time": convert_published_time(video.get('publishedTimeText', '')),
                    "username": username
                }
            }
            
            response = requests.post(
                f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/{POSTS_TABLE}",
                headers=headers,
                json=data
            )
            
            if response.status_code == 200:
                print(f"✓ Đã thêm video: {video['title']}")
            else:
                print(f"✗ Lỗi khi thêm video {video['title']}: {response.text}")

if __name__ == "__main__":
    main() 

Tìm thấy 4 users
Đã có 100 videos trong database

Đang xử lý channel: POCEnglish
Tìm thấy 138 videos mới
✓ Đã thêm video: Learn English Idioms and Vocabulary with Horror Movies!
✓ Đã thêm video: Formal or Informal English? Learn the Right Words
✓ Đã thêm video: Business English Vocabulary You Need to Know
✓ Đã thêm video: DO NOT say "I'm Sorry"! Apologize PROPERLY in English!
✓ Đã thêm video: I Hit 3M Subs and I'm Giving Back 50% Off My English Course!
✓ Đã thêm video: Speak English at the Doctor’s Office with Confidence
✓ Đã thêm video: 25 Must-Know English Idioms for Fluent Conversation!
✓ Đã thêm video: How Many of These 10 English Words Do You Know?
✓ Đã thêm video: Stop Overusing Simple English Words!
✓ Đã thêm video: English Learning Tips & Challenges: Stories from International Students
✓ Đã thêm video: English Vocabulary: Travel and Adventure
✓ Đã thêm video: Confusing English Phrasal Verbs? Not Anymore!
✓ Đã thêm video: What’s your English level? Take this test!
✓ Đã thêm vide

✓ Đã thêm video: 10 English Phrasal Verbs You Need To Know About Travelling
✓ Đã thêm video: 10 Most Common English Grammar Mistakes || Basic English Grammar
✓ Đã thêm video: English pronunciation: 12 contractions you MUST learn!
✓ Đã thêm video: Learn English vocabulary || Nature idioms
✓ Đã thêm video: 8 phrasal verbs in English to talk about studying and exams (super useful)
✓ Đã thêm video: 5 English idioms about fashion and clothing || English vocabulary in Milan's fashion district!
✓ Đã thêm video: English grammar: 10 Adjectives and adverbs with the same form!
✓ Đã thêm video: 20 adjectives to describe feelings!
✓ Đã thêm video: Adjectives or adverbs?? Let's make it clear!
✓ Đã thêm video: 10 words to talk about injuries in English language
✓ Đã thêm video: 5 very useful phrasal verbs to talk about arguments and disagreeing!
✓ Đã thêm video: 8 Travel idioms in English language and a trip to Venice! Part 2
✓ Đã thêm video: 5 English idioms about travelling, and a trip to Venice, I

✓ Đã thêm video: 13 THINGS MENTALLY STRONG PEOPLE DON'T DO by Amy Morin | Core Message
✓ Đã thêm video: OWN THE DAY, OWN YOUR LIFE by Aubrey Marcus | Core Message
✓ Đã thêm video: AWAKEN THE GIANT WITHIN by Tony Robbins | Core Message
✓ Đã thêm video: HOW TO STOP WORRYING AND START LIVING by Dale Carnegie | Core Message
✓ Đã thêm video: $100 STARTUP & 100 SIDE HUSTLES by Chris Guillebeau | Core Message
✓ Đã thêm video: MINI HABITS by Stephen Guise | Core Message
✓ Đã thêm video: INDISTRACTABLE by Nir Eyal | Core Message
✓ Đã thêm video: MAKE TIME by Jake Knapp and John Zeratsky | Core Message
✓ Đã thêm video: THE ART OF LEARNING by Josh Waitzkin | Core Message
✓ Đã thêm video: ULTRALEARNING by Scott Young | Core Message
✓ Đã thêm video: RANGE by David Epstein | Core Message
✓ Đã thêm video: MEDITATIONS by Marcus Aurelius | Core Message
✓ Đã thêm video: THE 5 ELEMENTS OF EFFECTIVE THINKING by Edward Burger & Michael Starbird
✓ Đã thêm video: SWITCH by Chip Heath and Dan Heath | Animated

Tìm thấy 119 videos mới
✓ Đã thêm video: Give me 14 minutes, And I'll Turn You Into An Academic Weapon
✓ Đã thêm video: My Honest Advice to Students Who Are Falling Behind
✓ Đã thêm video: You’re Not Dumb: How to Mindmap as a Beginner
✓ Đã thêm video: How to Remember Everything You Read
✓ Đã thêm video: How to Force Your Brain To Be Motivated (when you don’t feel like it)
✓ Đã thêm video: The Ultimate Guide to The Perfect Mindmap (6-Step Checklist)
✓ Đã thêm video: 5 Dimensions of Learning Every Student MUST Master
✓ Đã thêm video: 3 Levels of Mind Maps Every Student MUST Master
✓ Đã thêm video: 6 Levels of Thinking Every Student MUST Master
✓ Đã thêm video: How to Learn Any Skill Quickly (and Forever)
✓ Đã thêm video: How to Force Your Brain to Study (when you don't feel like it)
✓ Đã thêm video: How to Pick Your Perfect Career (part 2)
✓ Đã thêm video: How to Pick Your Perfect Career (The Ultimate Guide) - Part 1
✓ Đã thêm video: How to Beat Procrastination (Forever)
✓ Đã thêm video: